In [61]:
from glob import glob
from os import path
import seaborn as sns
import json
import os
import numpy as np
import pandas as pd
import re
from os.path import basename, splitext
import time
import pickle
import json

# from solver import Instance

from argparse import Namespace
import sys
sys.path.append('../solver')
# from solver_output import practice_print
from solver_rostering import run_roster_solver_objval_w_baseline

In [62]:
city = 'paris'

# fixed params
RM = 1.5
GM = 0.8
h_min = 32
h_max = 48
max_n_diff = 3

refresh = False

In [63]:
list_city_dt = []

for file_in in os.listdir(r'../instances'):
    base_file = splitext(basename(file_in))[0]
    base_file_split = base_file.split('_')
    file = base_file_split[0]+'_'+base_file_split[1]
    list_city_dt.append(file)

list_city_dt = list(set(list_city_dt))
list_city_dt.sort()

#Subset to frankfurt
list_city_dt_subset = []

for city_db in list_city_dt:
    if (city_db.find(city) > -1):
        list_city_dt_subset.append(city_db)

print(list_city_dt_subset)

['paris_db=0.50', 'paris_db=1.00', 'paris_db=2.00', 'paris_db=4.00']


In [64]:
def load_shift(shift_file, OC, RM, GM, model, max_n_shifts=None):
    with open(shift_file, 'r') as file:
        data = json.load(file)
        df_shifts = pd.DataFrame(data)
    df_shifts = df_shifts[(df_shifts['outsourcing_cost_multiplier']==OC)&(df_shifts['regional_multiplier']==RM)&(df_shifts['global_multiplier']==GM)]
    #fixed or flex
    if model in ['fixed','flex']:
        df_shifts = df_shifts[df_shifts['model']==model]
    #partflex
    else:
        df_shifts = df_shifts[(df_shifts['model']==model)&(df_shifts['max_n_shifts']==max_n_shifts)]
    df_shifts.reset_index(drop = True, inplace=True)
    df_shifts = df_shifts[['region','shifts_start','shifts_end']]
    dict_shifts = df_shifts.set_index('region').to_dict(orient='index')
    return dict_shifts

In [65]:
def load_workforce(workforce_file, OC, RM, GM, model, max_n_shifts=None):
    with open(workforce_file, 'r') as file:
        data = json.load(file)
        df_workforce = pd.DataFrame(data)
    df_workforce = df_workforce[(df_workforce['outsourcing_cost_multiplier']==OC)&(df_workforce['regional_multiplier']==RM)&(df_workforce['global_multiplier']==GM)]
    #fixed or flex
    if model in ['fixed','flex']:
        df_workforce = df_workforce[df_workforce['model']==model]
    #partflex
    else:
        df_workforce = df_workforce[(df_workforce['model']==model)&(df_workforce['max_n_shifts']==max_n_shifts)]
    df_workforce.reset_index(drop = True, inplace=True)
    return (int(df_workforce['workforce_size_region0'].tolist()[0])-1)


In [66]:
for base_file in list_city_dt_subset:

    #create inputs to run a trial workforce size
    weekday_ = f'{base_file}_dt=doublepeak.json'
    weekend_ = f'{base_file}_dt=uniform.json'

    instance_file_weekday = f"../instances/{weekday_}"
    instance_file_weekend = f"../instances/{weekend_}"

    shift_file_weekday = f"../shifts/{weekday_}"
    shift_file_weekend = f"../shifts/{weekend_}"

    expand_workforce_to_regions=True
    workforce_dict = {}

    new_shift_weekend = None
    new_shift_weekday = None
    new_workforce = None
    dict_results = {}

    for OC in [1.2, 1.5, 1.8, 2.0, 2.5]:

        new_shift_weekend = None
        new_shift_weekday = None
        new_workforce = None
        dict_results = {}

        for model in ['flex',  'partflex', 'fixed']:
            if model == 'partflex':
                for max_n_shifts in [4, 3, 2]:

                    old_results = dict_results
                    results_file = f"../raw_results/{base_file}_OC={OC}_model={model}_max_n_shifts={max_n_shifts}.pkl"

                    # Check if shift pattern is already solved
                    old_workforce = new_workforce
                    new_workforce = load_workforce(f'../workforce_size/{base_file}.json', OC, RM, GM, model, max_n_shifts)
                    old_shift_weekend = new_shift_weekend
                    old_shift_weekday = new_shift_weekday
                    new_shift_weekend = load_shift(shift_file_weekend, OC, RM, GM, model, max_n_shifts)
                    new_shift_weekday = load_shift(shift_file_weekday, OC, RM, GM, model, max_n_shifts)

                    if old_shift_weekend == new_shift_weekend and old_shift_weekday == new_shift_weekday and old_workforce == new_workforce:
                        old_results['baseline']['model']  = [model]
                        old_results['baseline']['max_n_shifts'] = max_n_shifts

                        print(f'Base: {base_file}, OC: {OC}, model: {model}-{max_n_shifts}, already solved. Saving previous results')

                        # Save previous results
                        with open(results_file, 'wb') as file:
                            pickle.dump(old_results, file)
                        continue
                    
                    start_time = time.time()
                    dict_baseline, dict_results = \
                        run_roster_solver_objval_w_baseline(model, instance_file_weekday, 
                                                            shift_file_weekday, instance_file_weekend, 
                                                            shift_file_weekend, workforce_dict, 
                                                            OC, RM, GM, h_min, h_max, max_n_diff, 
                                                            max_n_shifts, expand_workforce_to_regions=True)
                    
                    dict_results['baseline'] = dict_baseline
                    end_time = time.time()
                    run_time = (end_time - start_time)
                    
                    # Save results
                    with open(results_file, 'wb') as file:
                        pickle.dump(dict_results, file)
                    print(f'Base: {base_file}, OC: {OC}, model: {model}-{max_n_shifts}, run_time: {run_time}')

            else:

                old_results = dict_results
                results_file = f"../raw_results/{base_file}_OC={OC}_model={model}.pkl"

                # Check if shift pattern is already solved
                old_workforce = new_workforce
                new_workforce = load_workforce(f'../workforce_size/{base_file}.json', OC, RM, GM, model)

                old_shift_weekend = new_shift_weekend
                old_shift_weekday = new_shift_weekday
                new_shift_weekend = load_shift(shift_file_weekend, OC, RM, GM, model)
                new_shift_weekday = load_shift(shift_file_weekday, OC, RM, GM, model)

                if old_shift_weekend == new_shift_weekend and old_shift_weekday == new_shift_weekday and old_workforce == new_workforce:
                    old_results['baseline']['model']  = [model]
                    if 'max_n_shifts' in old_results['baseline']:
                        old_results['baseline']['max_n_shifts'] = None
                
                    print(f'Base: {base_file}, OC: {OC}, model: {model}, already solved. Saving previous results')
        
                    # Save previous results
                    with open(results_file, 'wb') as file:
                        pickle.dump(old_results, file)
                    continue
                
                results_file = f"../raw_results/{base_file}_OC={OC}_model={model}.pkl"
                # if not os.path.exists(results_file) or refresh:
                start_time = time.time()
                dict_baseline, dict_results = \
                    run_roster_solver_objval_w_baseline(model, instance_file_weekday, 
                                                        shift_file_weekday, instance_file_weekend, 
                                                        shift_file_weekend, workforce_dict, 
                                                        OC, RM, GM, h_min, h_max, max_n_diff, 
                                                        expand_workforce_to_regions=True)
                
                dict_results['baseline'] = dict_baseline
                end_time = time.time()
                run_time = (end_time - start_time)

                # Save results
                with open(results_file, 'wb') as file:
                    pickle.dump(dict_results, file)
                print(f'Base: {base_file}, OC: {OC}, model: {model}, run_time: {run_time}')


Base: lyon_db=0.50, OC: 1.2, model: flex, run_time: 7.386874198913574
Base: lyon_db=0.50, OC: 1.2, model: partflex-4, already solved. Saving previous results
Base: lyon_db=0.50, OC: 1.2, model: partflex-3, already solved. Saving previous results
Base: lyon_db=0.50, OC: 1.2, model: partflex-2, run_time: 4.241305112838745
Base: lyon_db=0.50, OC: 1.2, model: fixed, already solved. Saving previous results
Base: lyon_db=0.50, OC: 1.5, model: flex, run_time: 7.26092004776001
Base: lyon_db=0.50, OC: 1.5, model: partflex-4, already solved. Saving previous results
Base: lyon_db=0.50, OC: 1.5, model: partflex-3, already solved. Saving previous results
Base: lyon_db=0.50, OC: 1.5, model: partflex-2, run_time: 3.7577450275421143
Base: lyon_db=0.50, OC: 1.5, model: fixed, already solved. Saving previous results
Base: lyon_db=0.50, OC: 1.8, model: flex, run_time: 6.808063983917236
Base: lyon_db=0.50, OC: 1.8, model: partflex-4, already solved. Saving previous results
Base: lyon_db=0.50, OC: 1.8, mod

In [67]:

old_results

{'regions': [2, 3, 0, 1],
 'reg_areas': {2: ['69009', '69001', '69004', '69006', '69300'],
  3: ['69100', '69120', '69140'],
  0: ['69008', '69200', '69800', '69500'],
  1: ['69002', '69003', '69005', '69007']},
 'shifts': {(2, 0): [0, 4],
  (2, 1): [0, 4],
  (2, 2): [0, 4],
  (2, 3): [0, 4],
  (2, 4): [0, 4],
  (2, 5): [0, 4],
  (2, 6): [0, 4],
  (3, 0): [0, 4],
  (3, 1): [0, 4],
  (3, 2): [0, 4],
  (3, 3): [0, 4],
  (3, 4): [0, 4],
  (3, 5): [0, 4],
  (3, 6): [0, 4],
  (0, 0): [0, 4],
  (0, 1): [0, 4],
  (0, 2): [0, 4],
  (0, 3): [0, 4],
  (0, 4): [0, 4],
  (0, 5): [0, 4],
  (0, 6): [0, 4],
  (1, 0): [0, 4],
  (1, 1): [0, 4],
  (1, 2): [0, 4],
  (1, 3): [0, 4],
  (1, 4): [0, 4],
  (1, 5): [0, 4],
  (1, 6): [0, 4]},
 'days': [0, 1, 2, 3, 4, 5, 6],
 'employees': {2: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
  3: [16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33],
  0: [34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48],
  1: [49, 50, 51, 52,

In [68]:
workforce = load_workforce(f'../workforce_size/{base_file}.json', OC, RM, GM, model, max_n_shifts)

In [69]:
old_results['baseline']['model']

['fixed']